# PROCEDURE TABLE

TODO: Create a full list of procedure_concept_ids for IDEA4RC

In [99]:
import psycopg2
import pandas as pd
import random
from datetime import datetime, timedelta

In [100]:
dbname = "idea4rc"
user = "postgres"
password = "password"
host = "localhost"
port = "5432"

conn = psycopg2.connect(
    dbname=dbname,
    user=user,
    password=password,
    host=host,
    port=port
)

cur = conn.cursor()

In [101]:
def delete_table_data(table_name):
    try:
        # Connect to your postgres DB
        conn = psycopg2.connect(
            dbname=dbname,
            user=user,
            password=password,
            host=host,
            port=port
        )
        
        # Open a cursor to perform database operations
        cur = conn.cursor()
        
        # Execute a command: this deletes all data in the specified table
        cur.execute(f"DELETE FROM {table_name};")
        
        # Commit the changes
        conn.commit()
        
        # Close communication with the database
        cur.close()
        conn.close()
        
        print(f"Data from table {table_name} has been deleted.")
        
    except Exception as e:
        print(f"An error occurred: {e}")

### Generate data

In [102]:
def generate_fake_patients(num_patients):
    patients = []

    for person_id in range(1, num_patients + 1):
        patient = {
            'person_id': person_id,
            'gender_concept_id': random.choice([8507, 8532]),  # 8507: Male, 8532: Female
            'race_concept_id': random.choice([0]),  # Example race concept IDs
            'year_of_birth': random.randint(1920, 2020),
            'ethnicity_concept_id': random.choice([38003563, 38003564])  # Example ethnicity concept IDs
        }
        patients.append(patient)

    return pd.DataFrame(patients)

In [103]:
# Generate data
num_patients = 100
df_fake_patients = generate_fake_patients(num_patients)
df = df_fake_patients
df.head()

,person_id,gender_concept_id,race_concept_id,year_of_birth,ethnicity_concept_id
0,1,8507,0,1968,38003563
1,2,8532,0,1969,38003563
2,3,8532,0,1998,38003564
3,4,8507,0,1950,38003564
4,5,8507,0,1965,38003563


In [104]:
# df = pd.read_csv(r"./data/Patient_IDEA4RC.csv", delimiter=';')
# df.head(5)

In [105]:
delete_table_data('omopcdm3.person')

sql = """
    INSERT INTO omopcdm3.person (person_id, gender_concept_id, race_concept_id, year_of_birth, ethnicity_concept_id)
    VALUES (%s, %s, %s, %s, %s)
"""

cur.executemany(sql, zip(
    df.person_id, 
    df.gender_concept_id, 
    df.race_concept_id, 
    df.year_of_birth, 
    df.ethnicity_concept_id
    ))

conn.commit()

Data from table omopcdm3.person has been deleted.
